In [1]:
import json
import pandas as pd

from collections import defaultdict as dd
from character_mapping import MATRA # Odia language specific

In [2]:
# Path varibales
import os
input_data_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'input')
output_data_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'output')
wiki_file_path = os.path.join(input_data_path, 'cx-corpora.en2or.text_june_2019.json')

In [3]:
# File got from, https://dumps.wikimedia.org/other/contenttranslation/
with open(wiki_file_path, 'r', encoding='utf-8') as json_file:
    json_data = json.loads(json_file.read())

In [4]:
"""
# dict_keys(['id', 'sourceLanguage', 'targetLanguage', 'source', 'mt', 'target'])
#for i in range(2):
#    print(json_data[-i])

Sample JSON file:
{'id': '110783/mwCA', 'sourceLanguage': 'en', 'targetLanguage': 'or', 'source': {'content': 'View of Raj Path, from Raisina Hill, with India Gate at its end'}, 'mt': None, 'target': {'content': '+ ଅନୁବାଦ ଯୋଗକରନ୍ତୁ'}}
"""

"\n# dict_keys(['id', 'sourceLanguage', 'targetLanguage', 'source', 'mt', 'target'])\n#for i in range(2):\n#    print(json_data[-i])\n\nSample JSON file:\n{'id': '110783/mwCA', 'sourceLanguage': 'en', 'targetLanguage': 'or', 'source': {'content': 'View of Raj Path, from Raisina Hill, with India Gate at its end'}, 'mt': None, 'target': {'content': '+ ଅନୁବାଦ ଯୋଗକରନ୍ତୁ'}}\n"

In [5]:
pd_dict = dd(list)

In [6]:
for each_dict in json_data:
    try:
        or_content = each_dict.get('target', 'NA').get('content', '')
        if any(matra in or_content for matra in MATRA) and or_content.strip() != "+ ଅନୁବାଦ ଯୋଗକରନ୍ତୁ":
            pd_dict[each_dict.get('sourceLanguage', 'NA')].append(each_dict.get('source', 'NA').get('content', ''))
            pd_dict[each_dict.get('targetLanguage', 'NA')].append(or_content)
    except Exception as error_message:
        # Getting some issues while encoding need to look deeper
        pass

In [7]:
d = pd.DataFrame(data=pd_dict)
d.head()

,en,or
0,Otto H. Königsberger (13 October 1908 – 3 Janu...,ଓଟୋ କୋନିଙ୍ଗ୍ସବର୍ଗର (୧୨ ଅକ୍ଟୋବର ୧୯୦୮ – ୩ ଜାନୁଆର...
1,Bibliography,ପ୍ରକାଶନ
2,References,ଆଧାର
3,The small intestine or small bowel is the part...,ଅନ୍ତ The small intestine or small bowel is the...
4,External links,ବାହାର ଲିଙ୍କ


In [8]:
d.drop_duplicates(keep="last", inplace=True)

In [9]:
d.to_csv(os.path.join(output_data_path, 'output_partially_curated_june_2019.csv'), index=False)